## Leitura do arquivo

In [1]:
with open('dados/artigos.txt', 'r') as f:
  artigos = f.read()

##Tokenização

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
def separa_palavras(lista_tokens):
  lista_palavras = []
  for token in lista_tokens:
    if token.isalpha():
      lista_palavras.append(token)
  return lista_palavras

In [4]:
lista_tokens = nltk.tokenize.word_tokenize(artigos)
lista_palavras = separa_palavras(lista_tokens)
print('O número de palavras no corpus é {}.'.format(len(lista_palavras)))

O número de palavras no corpus é 403031.


In [5]:
def normalizacao(lista_palavras):
  lista_normalizada = []
  for palavra in lista_palavras:
    lista_normalizada.append(palavra.lower())
  return lista_normalizada

In [6]:
lista_normalizada = normalizacao(lista_palavras)
print('O número de palavras que o modelo vai aprender: {}.'.format(len(set(lista_normalizada))))

O número de palavras que o modelo vai aprender: 18464.


# Inserção de letras na palavra

In [7]:
def insere_letras(fatias):
    novas_palavras = []
    letras = 'abcdefghijklmnopqrstuvwxyzáâàãéêèẽíîìĩóôõòúûùũç'
    for E, D in fatias:
        for letra in letras:
            novas_palavras.append(E + letra + D)
    return novas_palavras
  
def gerador_palavras(palavra):
    fatias = []
    for i in range(len(palavra)+1):
        fatias.append((palavra[:i],palavra[i:]))
    palavras_geradas = insere_letras(fatias)
    return palavras_geradas

In [8]:
def corretor(palavra):
    palavras_geradas = gerador_palavras(palavra)
    palavra_correta = max(palavras_geradas, key=probabilidade)
    return palavra_correta

def probabilidade(palavra_gerada):
    return frequencia[palavra_gerada]/total_palavras

# Teste

In [9]:
frequencia = nltk.FreqDist(lista_normalizada)
total_palavras = len(lista_normalizada)
total_palavras

403031

In [10]:
palavra_exemplo = 'lgica'
corretor(palavra_exemplo)

'lógica'

# Avaliação

In [11]:
def cria_dados_teste(nome_arquivo):
  lista_palavras_teste = []
  f = open(nome_arquivo, 'r')
  for linha in f:
    correta, errada = linha.split()
    lista_palavras_teste.append((correta, errada))
  f.close()
  return lista_palavras_teste

In [12]:
def avaliador(testes):
    numero_palavras = len(testes)
    acertou = 0
    for correta, errada in testes:
        palavra_corrigida = corretor(errada)
        if palavra_corrigida == correta:
            acertou += 1
    taxa_acerto = round(acertou*100/numero_palavras, 2)
    print('Taxa de acerto: {}%'.format(taxa_acerto))

lista_teste = cria_dados_teste('dados/palavras.txt')
avaliador(lista_teste)

Taxa de acerto: 1.08%


#Exclusão de letra da palavra

In [13]:
def deletando_caracteres(fatias):
  novas_palavras = []
  for E, D in fatias:
    novas_palavras.append(E + D[1:])
  return novas_palavras

In [14]:
def gerador_palavras(palavra):
    fatias = []
    for i in range(len(palavra)+1):
        fatias.append((palavra[:i],palavra[i:]))
    palavras_geradas = insere_letras(fatias)
    palavras_geradas += deletando_caracteres(fatias)
    return palavras_geradas

In [15]:
palavra_exemplo = 'lóigica'
palavras_geradas = gerador_palavras(palavra_exemplo)

In [16]:
avaliador(lista_teste)

Taxa de acerto: 41.4%


# Troca de letras

In [17]:
def troca_letra(fatias):
  novas_palavras= []
  letras = 'abcdefghijklmnopqrstuvwxyzáâàãéêèẽíîìĩóôõòúûùũç'
  for E, D in fatias:
    for letra in letras:
      novas_palavras.append(E + letra + D[1:])
  return novas_palavras

def gerador_palavras(palavra):
    fatias = []
    for i in range(len(palavra)+1):
        fatias.append((palavra[:i],palavra[i:]))
    palavras_geradas = insere_letras(fatias)
    palavras_geradas += deletando_caracteres(fatias)
    palavras_geradas += troca_letra(fatias)
    return palavras_geradas

In [18]:
palavra_exemplo = 'lígica'
palavras_geradas = gerador_palavras(palavra_exemplo)

#Letras invertidas

In [19]:
def invertendo_caracteres(fatias):
  novas_palavras = []
  for E, D in fatias:
    if(len(D) > 1):
      novas_palavras.append(E + D[1] + D[0] + D[2:])
  return novas_palavras

In [20]:
palavra_exemplo = 'lógiac'
palavras_geradas = gerador_palavras(palavra_exemplo)

In [21]:
avaliador(lista_teste)

Taxa de acerto: 76.34%


# Palavras desconhecidas ao vocabulário

In [22]:
def avaliador(testes, vocabulario):
    numero_palavras = len(testes)
    acertou, desconhecida = 0, 0
    for correta, errada in testes:
        palavra_corrigida = corretor(errada)
        if palavra_corrigida == correta:
            acertou += 1
        else:
          desconhecida += (correta not in vocabulario)
    taxa_acerto = round(acertou*100/numero_palavras, 2)
    taxa_palavras_desconhecidas = round(desconhecida*100/numero_palavras, 2)
    print('Taxa de acerto: {}%'.format(taxa_acerto))
    print('Taxa de palavras desconhecidas: {}%'.format(taxa_palavras_desconhecidas))

In [23]:
vocabulario = set(lista_normalizada)
lista_teste = cria_dados_teste('dados/palavras.txt')
avaliador(lista_teste, lista_normalizada)

Taxa de acerto: 76.34%
Taxa de palavras desconhecidas: 6.99%


# Gerador de palavras turbinado (corrige até duas distâncias da palavra correta)

In [24]:
def gerador_turbinado(palavras_geradas):
  novas_palavras = []
  for palavra in palavras_geradas:
    novas_palavras += gerador_palavras(palavra)
  return novas_palavras

In [25]:
palavras_g = gerador_turbinado(gerador_palavras('lóiigica'))
'lógica' in palavras_g

True

In [26]:
len(palavras_g)

774915

In [27]:
def novo_corretor(palavra):
  palavras_geradas = gerador_palavras(palavra)
  palavras_geradas_turbinado = gerador_turbinado(palavras_geradas)
  todas_palavras = set(palavras_geradas + palavras_geradas_turbinado)
  candidatos = [palavra]
  for palavra in todas_palavras:
    if palavra in vocabulario:
      candidatos.append(palavra)
  palavra_correta = max(candidatos, key=probabilidade)
  return palavra_correta

# Avaliação do novo corretor

In [31]:
def avaliador(testes, vocabulario):
    numero_palavras = len(testes)
    acertou, desconhecida = 0, 0
    for correta, errada in testes:
        palavra_corrigida = corretor(errada)
        desconhecida += (correta not in vocabulario)
        if palavra_corrigida == correta:
            acertou += 1
    taxa_acerto = round(acertou*100/numero_palavras, 2)
    taxa_palavras_desconhecidas = round(desconhecida*100/numero_palavras, 2)
    print('Taxa de acerto: {}%'.format(taxa_acerto))
    print('Taxa de palavras desconhecidas: {}%'.format(taxa_palavras_desconhecidas))

volabulario = set(lista_normalizada)
avaliador(lista_teste, vocabulario)

Taxa de acerto: 76.34%
Taxa de palavras desconhecidas: 6.99%


In [39]:
palavra = 'lóiigica'
print(novo_corretor(palavra))
print(corretor(palavra))

lógica
alóiigica
